In [1]:
%%capture
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin
from google.colab import files

In [2]:
files.upload()
data = pd.read_csv("data_CBR.csv", sep=";")
dataset = data.values

Saving data_CBR.csv to data_CBR.csv


In [3]:
class tailored_scaler(TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return np.log10(X+2.71828)
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [4]:
tailored_scaler = tailored_scaler()
dataset = tailored_scaler.fit_transform(dataset)
scaler = StandardScaler()
dataset = scaler.fit_transform(dataset)
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
np.take(dataset,np.random.permutation(dataset.shape[0]),axis=0,out=dataset)
total_size = len(dataset)
train_size = round(total_size*0.8)
validation_size = round(total_size*0.2)
train_dataset = dataset[0:train_size]
validation_dataset = dataset[train_size:total_size]
input_train, output_train = train_dataset[:,0:6], train_dataset[: ,6:9]
input_validation, output_validation = validation_dataset[:,0:6], validation_dataset[: ,6:9]

In [5]:
def build_model():
  model = keras.Sequential([
    layers.Dense(3, activation="relu", input_shape=[6]),
    layers.Dense(8, activation="sigmoid"),
    layers.Dense(2),
    layers.Dense(3)])
  lr_schedule = optimizers.schedules.ExponentialDecay(initial_learning_rate=0.013, decay_steps=250, decay_rate=0.98)
  optimizer = optimizers.RMSprop(momentum=0.001, learning_rate=lr_schedule, epsilon=1e-6, name="RMSprop")
  model.compile(loss="mse", optimizer=optimizer, metrics=["msle"])
  return model
model = build_model()
epochs = 2000
history = model.fit(input_train, output_train, epochs=epochs, validation_data=(input_validation, output_validation), verbose=0)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 21        
                                                                 
 dense_1 (Dense)             (None, 8)                 32        
                                                                 
 dense_2 (Dense)             (None, 2)                 18        
                                                                 
 dense_3 (Dense)             (None, 3)                 9         
                                                                 
Total params: 80 (320.00 Byte)
Trainable params: 80 (320.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
